In [ ]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 36.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366450 sha256=e5a56e1e6ac2939178221a63b013b8e677df92e4af9729b40a16eee5577aeafb
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [ ]:
import pandas as pd
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate
pd.set_option('display.max_columns', None)

Adım 1: Veri Setinin Hazırlanması


Adım 2: Modelleme

Adım 3: Model Tuning

Adım 4: Final Model ve Tahmin

In [ ]:
movie = pd.read_csv('/content/movie.csv')
rating = pd.read_csv('/content/rating.csv')
df = movie.merge(rating, how="left", on="movieId")

In [ ]:
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


In [ ]:
movie_ids = [130219, 356, 4422, 541]

In [ ]:
movies = ["The Dark Knight (2011)",
          "Cries and Whispers (Viskningar och rop) (1972)",
          "Forrest Gump (1994)",
          "Blade Runner (1982)"]


In [ ]:
sample_df = df[df.movieId.isin(movie_ids)]

In [ ]:
sample_df.head()

,movieId,title,genres,userId,rating,timestamp
70505,356,Forrest Gump (1994),Comedy|Drama|Romance|War,4.0,4.0,1996-08-24 09:28:42
70506,356,Forrest Gump (1994),Comedy|Drama|Romance|War,7.0,4.0,2002-01-16 19:02:55
70507,356,Forrest Gump (1994),Comedy|Drama|Romance|War,8.0,5.0,1996-06-05 13:44:19
70508,356,Forrest Gump (1994),Comedy|Drama|Romance|War,9.0,4.0,2001-07-01 20:26:38
70509,356,Forrest Gump (1994),Comedy|Drama|Romance|War,10.0,3.0,1999-11-25 02:32:02


In [ ]:
sample_df.shape

(2793, 6)

In [ ]:
user_movie_df = sample_df.pivot_table(index=["userId"],
                                      columns=["title"],
                                      values="rating")

In [ ]:

user_movie_df.shape

(2200, 4)

In [ ]:
reader = Reader(rating_scale=(1, 5))

In [ ]:
data = Dataset.load_from_df(sample_df[['userId',
                                       'movieId',
                                       'rating']], reader)


Adım 2: Modelleme

In [ ]:
trainset, testset = train_test_split(data, test_size=.25)
svd_model = SVD()
svd_model.fit(trainset)
predictions = svd_model.test(testset)

In [ ]:
accuracy.rmse(predictions)

RMSE: 0.9808


0.9807522831462147

In [ ]:
svd_model.predict(uid=1.0, iid=541, verbose=True)

user: 1.0        item: 541        r_ui = None   est = 4.07   {'was_impossible': False}


Prediction(uid=1.0, iid=541, r_ui=None, est=4.067656294778928, details={'was_impossible': False})

In [ ]:
svd_model.predict(uid=1.0, iid=356, verbose=True)

user: 1.0        item: 356        r_ui = None   est = 4.13   {'was_impossible': False}


Prediction(uid=1.0, iid=356, r_ui=None, est=4.1345214410892375, details={'was_impossible': False})

In [ ]:
sample_df[sample_df["userId"] == 1]

,movieId,title,genres,userId,rating,timestamp
103343,541,Blade Runner (1982),Action|Sci-Fi|Thriller,1.0,4.0,2005-04-02 23:30:03


Adım 3: Model Tuning

In [ ]:
param_grid = {'n_epochs': [5, 10, 20],
              'lr_all': [0.002, 0.005, 0.007]}

In [ ]:
gs = GridSearchCV(SVD,
                  param_grid,
                  measures=['rmse', 'mae'],
                  cv=3,
                  n_jobs=-1,
                  joblib_verbose=True)

In [ ]:
gs.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    1.2s finished


In [ ]:
gs.best_score['rmse']

0.9264394785390074

In [ ]:
gs.best_params['rmse']

{'n_epochs': 5, 'lr_all': 0.002}

Adım 4: Final Model ve Tahmin

In [ ]:
dir(svd_model)
svd_model.n_epochs

svd_model = SVD(**gs.best_params['rmse'])

data = data.build_full_trainset()
svd_model.fit(data)

svd_model.predict(uid=1.0, iid=541, verbose=True)


user: 1.0        item: 541        r_ui = None   est = 4.27   {'was_impossible': False}


Prediction(uid=1.0, iid=541, r_ui=None, est=4.270512895013597, details={'was_impossible': False})